<a href="https://colab.research.google.com/github/danijak/testing/blob/master/Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
# loading the MNIST dataset from keras
data = keras.datasets.mnist 

In [4]:
# training & testing split
(train_images, train_labels), (test_images, test_labels) = data.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
# Input Normalize to get it into 0-1
train_images = train_images/255.0
test_images = test_images/255.0

In [30]:
# model
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(28,28)))
model.add(keras.layers.Reshape(target_shape=(28,28,1)))
model.add(keras.layers.Conv2D(12,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2028)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________


In [34]:
#Train the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(x=train_images, y=train_labels, batch_size=32, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2811 - accuracy: 0.9190 - val_loss: 0.2245 - val_accuracy: 0.9363
Epoch 2/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2790 - accuracy: 0.9192 - val_loss: 0.2250 - val_accuracy: 0.9370
Epoch 3/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2768 - accuracy: 0.9196 - val_loss: 0.2223 - val_accuracy: 0.9375
Epoch 4/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2750 - accuracy: 0.9208 - val_loss: 0.2222 - val_accuracy: 0.9360
Epoch 5/5
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2732 - accuracy: 0.9213 - val_loss: 0.2218 - val_accuracy: 0.9372


In [36]:
# Evaluate
print(model.evaluate(x=test_images, y=test_labels))


313/313 [==============================] - 2s 5ms/step - loss: 0.2637 - accuracy: 0.9245
[0.26367828249931335, 0.9244999885559082]


In [39]:
! pip install tensorflow_model_optimization

     |████████████████████████████████| 174kB 2.8MB/s 


In [40]:
# Pre trained model with pruning
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()


Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_4 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense_2  (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


In [42]:
import tempfile

In [43]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
  1/422 [..............................] - ETA: 0s - loss: 0.3368 - accuracy: 0.9141WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
422/422 [==============================] - 13s 31ms/step - loss: 0.2774 - accuracy: 0.9215 - val_loss: 0.2587 - val_accuracy: 0.9358
Epoch 2/2
422/422 [==============================] - 13s 30ms/step - loss: 0.3226 - accuracy: 0.9169 - val_loss: 0.2761 - val_accuracy: 0.9328


In [45]:
print(model_for_pruning.evaluate(x=test_images, y=test_labels))

313/313 [==============================] - 2s 5ms/step - loss: 0.3080 - accuracy: 0.9183
[0.3080199956893921, 0.9182999730110168]
